### Import

In [47]:
import numpy as np
import pandas as pd 
from collections import Counter
import os
import glob
import copy
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

### Read every cutoffs from each method

In [48]:
datasets_1 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/gmm/cutoffs/' + "/*."+'csv'))]
files_1 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/gmm/cutoffs/'))]

datasets_2 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/kmeans/cutoffs/' + "/*."+'csv'))]
files_2 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/kmeans/cutoffs/'))]

datasets_3 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/roc/cutoffs/' + "/*."+'csv'))]
files_3 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/roc/cutoffs/'))]

datasets_4 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/tertile/cutoffs/' + "/*."+'csv'))]
files_4 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/tertile/cutoffs/'))]

datasets_5 = [pd.read_csv(file, index_col=0, low_memory=False) for file in sorted(glob.glob('../results/bootstrap/mean_sd/cutoffs/' + "/*."+'csv'))]
files_5 = [file.split(".")[0] for file in sorted(os.listdir('../results/bootstrap/mean_sd/cutoffs/'))]

In [49]:
# make a dictionary that contains all cutoffs as a dataframe
gmm = dict()
kmeans = dict()
roc = dict()
tertile = dict()
mean_sd = dict()

for file_, name_ in zip(datasets_1, files_1):
    gmm[name_] = file_
    
for file_, name_ in zip(datasets_2, files_2):
    kmeans[name_] = file_
    
for file_, name_ in zip(datasets_3, files_3):
    roc[name_] = file_
    
for file_, name_ in zip(datasets_4, files_4):
    tertile[name_] = file_
    
for file_, name_ in zip(datasets_5, files_5):
    mean_sd[name_] = file_

In [50]:
#rename columns before combining the dataframes 
for i in kmeans:
    for column in kmeans[i].columns:
        kmeans[i].rename(columns={column: column.split('_')[0]}, inplace=True)

### Preprocess the cutoffs
* make a dictionary with all the cutoffs
    * keys would be the cohort names
    * values would be a dataframe with all the cutoffs of CSF biomarkers

In [51]:
def cohort_based_cutoffs(method_dfs):
    """ """

#     biomarkers = method_dfs['gmm_cutoffs_0'].columns.to_list()
    cutpoints = {i: pd.DataFrame(index=list(range(1000)), 
                columns=method_dfs[list(method_dfs.keys())[0]].columns) 
                for i in method_dfs[list(method_dfs.keys())[0]].index}

    for bioma in cutpoints['ADNI'].columns:
        
        for cohort in method_dfs[list(method_dfs.keys())[0]].index:
            
            for i in method_dfs:
                cutpoints[cohort].loc[int(i.split('_')[-1]), bioma] = method_dfs[i].loc[cohort, bioma]
                
    return cutpoints

In [52]:
gmm_new = cohort_based_cutoffs(gmm)
kmeans_new = cohort_based_cutoffs(kmeans)
roc_new = cohort_based_cutoffs(roc)
tertile_new = cohort_based_cutoffs(tertile)
mean_sd_new = cohort_based_cutoffs(mean_sd)

### Calculate the confidence intervals of CSF cutoffs for each method 

In [53]:
# results = {i: pd.DataFrame(index=['GMM', 'ROC', 'K-Means', 'Tertile', 'Mean ±2 SD'], 
#            columns=gmm[list(gmm.keys())[0]].columns) 
#             for i in gmm[list(gmm.keys())[0]].index}

results = {i: pd.DataFrame(index=gmm[list(gmm.keys())[0]].index, 
           columns=gmm[list(gmm.keys())[0]].columns) 
            for i in ['GMM', 'K-Means', 'Tertile', 'ROC', 'Mean ±2 SD']}

results_percentage = {i: pd.DataFrame(index=gmm[list(gmm.keys())[0]].index, 
           columns=gmm[list(gmm.keys())[0]].columns) 
            for i in ['GMM', 'K-Means', 'Tertile', 'ROC', 'Mean ±2 SD']}

In [54]:
def mean_confidence_interval(methodologies, results, confidence=0.8):
    
    
    for method in methodologies:    
        datasets_ = methodologies[method]

        for i in datasets_:

            for col in datasets_[i].columns:
    #             data = datasets_[i][col].to_list()
                a = 1.0 * np.array(datasets_[i][col])
                n = len(a)
                m, se = np.nanmean(a, 0), stats.sem(a, nan_policy="omit")
                h = se * stats.t.ppf((1 + confidence) / 2., n-1)
#                 results[i].loc[method, col] = f'{round(m, 2)} [{round(m-h, 2)}, {round(m+h, 2)}]'
                results[method].loc[i, col] = f'{round(m, 1)} [{round(m-h, 1)}, {round(m+h, 1)}]'
                results_percentage[method].loc[i, col] = f'{round((((m+h) - (m-h))/m) * 100, 2)} %'

In [55]:
methodologies = {'GMM': gmm_new, 'K-Means': kmeans_new, 'ROC': roc_new, 'Tertile': tertile_new, 
                 'Mean ±2 SD': mean_sd_new}

In [56]:
mean_confidence_interval(methodologies, results)

In [57]:
pd.concat(results, axis=1).to_csv("../results/bootstrap/confidence_intervals_80.csv")

In [58]:
results = pd.concat(results, axis=1)

In [76]:
# for i,j in enumerate(results.columns): 
#     print(j[1].split(" in CSF")[0] + '_' + j[0])

In [59]:
results.replace({np.nan: "-"})

GMM                        \
                 A-beta 1-42 in CSF           pTau in CSF   
ADNI           986.9 [986.3, 987.5]     34.7 [34.6, 34.7]   
EPAD        1035.7 [1035.2, 1036.1]     27.8 [27.8, 27.8]   
AIBL           749.0 [745.2, 752.8]     87.8 [87.1, 88.5]   
ARWIBO         589.2 [586.4, 592.0]  155.0 [150.6, 159.5]   
EDSD           782.2 [778.2, 786.2]  119.2 [118.2, 120.2]   
PREVENT-AD  1187.4 [1168.6, 1206.2]     71.1 [70.6, 71.7]   
PharmaCog      792.9 [791.8, 794.1]     93.0 [92.8, 93.2]   
NACC_ELISA     594.1 [593.1, 595.1]     81.7 [81.1, 82.2]   
EMIF_ELISA     742.1 [741.5, 742.7]     82.1 [82.0, 82.2]   
NACC_XMAP      298.7 [297.6, 299.8]     60.8 [60.2, 61.4]   
EMIF_XMAP      386.9 [385.2, 388.6]     51.6 [51.3, 51.9]   
DOD-ADNI    1412.8 [1401.1, 1424.6]     26.6 [26.5, 26.6]   
JADNI          414.7 [413.3, 416.2]     61.1 [60.9, 61.3]   

                                                  K-Means                     \
                     tTau in CSF       A-beta 1-42 in CSF        pTau in CSF   
ADNI        356.9 [356.3, 357.4]     972.6 [971.9, 973.2]  28.1 [28.1, 28.1]   
EPAD        307.0 [306.8, 307.3]  1033.3 [1032.9, 1033.8]  19.8 [19.8, 19.9]   
AIBL        604.5 [599.6, 609.4]     684.0 [682.2, 685.8]  77.7 [77.2, 78.1]   
ARWIBO      531.6 [528.4, 534.7]     523.8 [522.4, 525.2]  69.1 [69.0, 69.3]   
EDSD        584.8 [579.2, 590.5]     741.8 [738.1, 745.5]  86.7 [86.3, 87.2]   
PREVENT-AD  466.5 [460.4, 472.6]  1146.4 [1144.9, 1147.9]  50.8 [50.6, 50.9]   
PharmaCog   773.4 [758.6, 788.2]     762.2 [760.4, 763.9]  68.6 [68.2, 69.0]   
NACC_ELISA  564.6 [556.7, 572.6]     586.1 [585.3, 586.8]  61.6 [61.5, 61.8]   
EMIF_ELISA  526.6 [525.0, 528.1]     697.7 [696.1, 699.3]  64.9 [64.8, 64.9]   
NACC_XMAP      94.5 [94.1, 95.0]     290.7 [289.2, 292.2]  40.1 [40.1, 40.2]   
EMIF_XMAP   227.7 [223.2, 232.2]     363.3 [362.6, 363.9]  37.5 [37.5, 37.6]   
DOD-ADNI    304.0 [302.8, 305.1]  1273.5 [1269.3, 1277.6]  22.4 [22.3, 22.4]   
JADNI       133.9 [133.0, 134.7]     392.5 [391.3, 393.8]  59.9 [59.7, 60.0]   

                                                  Tertile                     \
                     tTau in CSF       A-beta 1-42 in CSF        pTau in CSF   
ADNI        288.2 [288.0, 288.4]     833.3 [831.9, 834.6]  21.8 [21.8, 21.9]   
EPAD        223.7 [223.6, 223.9]     884.5 [883.9, 885.1]  19.9 [19.8, 19.9]   
AIBL        530.6 [525.9, 535.4]                        -                  -   
ARWIBO      432.1 [429.9, 434.2]                        -                  -   
EDSD        547.6 [544.6, 550.6]                        -                  -   
PREVENT-AD  308.8 [307.4, 310.2]  1098.7 [1097.5, 1100.0]  53.0 [52.9, 53.1]   
PharmaCog   466.9 [465.4, 468.4]                        -                  -   
NACC_ELISA  492.1 [490.8, 493.3]     625.5 [624.3, 626.8]  48.3 [48.2, 48.3]   
EMIF_ELISA  432.5 [432.0, 433.1]     541.0 [540.2, 541.8]  52.3 [52.2, 52.4]   
NACC_XMAP      70.2 [70.1, 70.4]     248.5 [248.0, 249.0]  36.5 [36.4, 36.5]   
EMIF_XMAP   128.9 [128.5, 129.3]     446.7 [445.5, 447.8]  31.4 [31.2, 31.5]   
DOD-ADNI    249.8 [249.1, 250.4]     928.4 [925.8, 931.0]  19.8 [19.8, 19.9]   
JADNI       129.0 [128.3, 129.7]     392.7 [391.5, 393.9]  38.5 [38.5, 38.6]   

                                                   ROC                     \
                     tTau in CSF    A-beta 1-42 in CSF        pTau in CSF   
ADNI        240.1 [239.7, 240.4]  819.4 [817.1, 821.8]  23.6 [23.5, 23.6]   
EPAD        228.2 [228.0, 228.4]  684.7 [682.0, 687.5]  23.9 [23.7, 24.2]   
AIBL                           -                     -                  -   
ARWIBO                         -                     -                  -   
EDSD                           -                     -                  -   
PREVENT-AD  303.3 [302.8, 303.8]                     -                  -   
PharmaCog                      -                     -                  -   
NACC_ELISA  371.3 [37

In [60]:
pd.concat(results_percentage, axis=1)

GMM                                    K-Means  \
           A-beta 1-42 in CSF pTau in CSF tTau in CSF A-beta 1-42 in CSF   
ADNI                   0.12 %      0.42 %      0.31 %             0.13 %   
EPAD                   0.09 %      0.22 %      0.17 %             0.09 %   
AIBL                   1.01 %      1.56 %      1.62 %             0.53 %   
ARWIBO                 0.94 %      5.78 %       1.2 %             0.53 %   
EDSD                   1.02 %       1.7 %      1.93 %              1.0 %   
PREVENT-AD             3.17 %      1.61 %      2.61 %             0.27 %   
PharmaCog               0.3 %       0.4 %      3.82 %             0.45 %   
NACC_ELISA             0.34 %       1.3 %      2.83 %             0.26 %   
EMIF_ELISA             0.16 %       0.3 %      0.59 %             0.46 %   
NACC_XMAP              0.75 %      1.96 %      1.01 %             1.02 %   
EMIF_XMAP              0.89 %      1.15 %      3.93 %             0.34 %   
DOD-ADNI               1.66 %      0.59 %      0.74 %             0.65 %   
JADNI                  0.69 %      0.69 %      1.26 %             0.63 %   

                                              Tertile                          \
           pTau in CSF tTau in CSF A-beta 1-42 in CSF pTau in CSF tTau in CSF   
ADNI            0.17 %      0.17 %             0.32 %      0.33 %      0.31 %   
EPAD            0.14 %      0.13 %             0.14 %      0.18 %      0.18 %   
AIBL            1.18 %       1.8 %                NaN         NaN         NaN   
ARWIBO          0.47 %       1.0 %                NaN         NaN         NaN   
EDSD            1.05 %       1.1 %                NaN         NaN         NaN   
PREVENT-AD      0.59 %      0.91 %             0.23 %      0.34 %      0.33 %   
PharmaCog       1.12 %      0.64 %                NaN         NaN         NaN   
NACC_ELISA      0.45 %       0.5 %             0.41 %      0.36 %      0.59 %   
EMIF_ELISA      0.18 %      0.25 %              0.3 %      0.26 %      0.37 %   
NACC_XMAP       0.42 %      0.42 %             0.41 %      0.25 %       0.4 %   
EMIF_XMAP       0.35 %      0.57 %             0.51 %      1.01 %      0.89 %   
DOD-ADNI        0.63 %      0.53 %             0.56 %      0.43 %       0.4 %   
JADNI           0.54 %      1.08 %              0.6 %      0.29 %      0.46 %   

                          ROC                                 Mean ±2 SD  \
           A-beta 1-42 in CSF pTau in CSF tTau in CSF A-beta 1-42 in CSF   
ADNI                   0.58 %      0.56 %       0.5 %             0.92 %   
EPAD                    0.8 %      1.97 %      2.22 %             0.35 %   
AIBL                      NaN         NaN         NaN                NaN   
ARWIBO                    NaN         NaN         NaN                NaN   
EDSD                      NaN         NaN         NaN                NaN   
PREVENT-AD                NaN         NaN         NaN             0.87 %   
PharmaCog                 NaN         NaN         NaN                NaN   
NACC_ELISA             0.96 %      0.76 %      1.19 %              0.8 %   
EMIF_ELISA             0.99 %      0.98 %       0.8 %             1.17 %   
NACC_XMAP              0.31 %       0.6 %      1.18 %             5.67 %   
EMIF_XMAP              0.75 %      1.12 %      0.74 %              1.6 %   
DOD-ADNI                  NaN         NaN         NaN             1.74 %   
JADNI                  0.55 %      0.71 %      0.27 %              1.2 %   

                                    
           pTau in CSF tTau in CSF  
ADNI            0.35 %      0.29 %  
EPAD            0.28 %      0.22 %  
AIBL               NaN         NaN  
ARWIBO             NaN         NaN  
EDSD               NaN         NaN  
PREVENT-AD      0.57 %      0.84 %  
PharmaCog          NaN         NaN  
NACC_ELISA      0.73 %      0.61 %  
EMIF_ELISA       0.7 %      1.67 %  
NACC_XMAP       0.78 %      0.63 %  
EMIF_XMAP        0.9 %      0.55 %  
DOD-ADNI        0.55 %      0.44 %  
JADNI     

In [61]:
results_percentage['GMM']

,A-beta 1-42 in CSF,pTau in CSF,tTau in CSF
ADNI,0.12 %,0.42 %,0.31 %
EPAD,0.09 %,0.22 %,0.17 %
AIBL,1.01 %,1.56 %,1.62 %
ARWIBO,0.94 %,5.78 %,1.2 %
EDSD,1.02 %,1.7 %,1.93 %
PREVENT-AD,3.17 %,1.61 %,2.61 %
PharmaCog,0.3 %,0.4 %,3.82 %
NACC_ELISA,0.34 %,1.3 %,2.83 %
EMIF_ELISA,0.16 %,0.3 %,0.59 %
NACC_XMAP,0.75 %,1.96 %,1.01 %


In [13]:
round(596.25, 1)

596.2

In [14]:
import numpy as np
np.round(596.25, 1)

596.2